In [24]:
import torch
import torch.nn.functional as F
from torch import nn

In [ ]:

import kagglehub
import os

In [ ]:
# Download latest version
path = kagglehub.dataset_download("grafstor/simple-dialogs-for-chatbot")
files = os.listdir(path)
with open(os.path.join(path, files[0]), 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
n_embd = 384  # Size of the embedding vectors
print("Number of unique characters:", vocab_size)
print(''.join(chars))

Number of unique characters: 65

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [61]:
max_iterations = 1000
eval_interval = 500
learning_rate = 3e-4
batch_size = 64  # Number of sequences in a batch
block_size = 256  # Size of the context window
eval_iters = 200
n_embd = 384
n_head = 6  # Number of attention heads
n_layer = 6  # Number of transformer blocks
dropout = 0.2  # Dropout rate

In [4]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[s] for s in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hello"))
print(decode(encode("hello")))

[46, 43, 50, 50, 53]
hello


In [5]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)  # Print shape and dtype of the tensor
print(data[:1000])  # Print the first 1000 elements of the tensor to verify encoding

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [6]:
n = int(0.9 * len(data))  # 90% for training
train_data = data[:n]
val_data = data[n:]

In [7]:
block_size = 8  # Size of the context window
x = train_data[:block_size]  # Input sequence
y = train_data[1:block_size+1]  # Target sequence (next character)
for i in range(block_size):
    context = x[:i+1]
    target = y[i]
    print(f"when input is {context}, the target is {target}")


when input is tensor([18]), the target is 47
when input is tensor([18, 47]), the target is 56
when input is tensor([18, 47, 56]), the target is 57
when input is tensor([18, 47, 56, 57]), the target is 58
when input is tensor([18, 47, 56, 57, 58]), the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is 58


In [8]:
torch.manual_seed(1337)  # For reproducibility
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B, T, C)  # Random input tensor
print(x.shape)  # Should print torch.Size([4, 8, 2])

torch.Size([4, 8, 2])


In [19]:
xbow = torch.zeros((B, T, C))  # Initialize the output tensor
for b in range(B):
    for t in range(T):
        x_prev = x[b, :t+1]  # Get the sequence up to time t
        xbow[b, t] = torch.mean(x_prev, dim=0)  # Compute the mean across the time dimension

In [22]:
wei = torch.tril(torch.ones(T, T))  # Lower triangular matrix
wei = wei / wei.sum(dim=1, keepdim=True)  # Normalize to get weights
xbow2 = wei @ x  # Weighted average using the weights
torch.allclose(xbow, xbow2, atol=1e-6)  # Check if both methods yield the same result


True

In [25]:
tril = torch.tril(torch.ones(T, T))  # Lower triangular matrix
wei = torch.zeros((T, T))  # Initialize weights
wei = wei.masked_fill(tril == 0, float('-inf'))  # Set upper triangular part to -inf
wei = F.softmax(wei, dim=-1)  # Apply softmax to get probabilities
xbow3 = wei @ x  # Weighted average using the weights
torch.allclose(xbow, xbow3, atol=1e-6)  # Check if both methods yield the same result

True

In [ ]:
torch.manual_seed(1337)  # For reproducibility
B, T, C = 4, 8, 32  # Batch size, time steps, channels
x = torch.randn(B, T, C)  # Random input tensor

head_size = 16  # Size of each head
key = nn.Linear(C, head_size, bias=False)  # Linear layer for keys
query = nn.Linear(C, head_size, bias=False)  # Linear layer for queries
value = nn.Linear(C, head_size, bias=False)  # Linear layer for values
k = key(x)  # Compute keys
q = query(x)  # Compute queries
wei = q @ k.transpose(-2, -1)  # Compute attention scores

tril = torch.tril(torch.ones(T, T))  # Lower triangular matrix
#wei = torch.zeros((T, T))  # Initialize weights
wei = wei.masked_fill(tril == 0, float('-inf'))  # Set upper triangular part to -inf
wei = F.softmax(wei, dim=-1)  # Apply softmax to get probabilities
#out = wei @ x  # Weighted average using the weights
v = value(x)  # Compute values
out = wei @ v  # Weighted average using the values
print(out.shape)  # Should print torch.Size([4, 8, 32])

torch.Size([4, 8, 16])


In [ ]:
torch.manual_seed(1337)  # For reproducibility
block_size = 256  # Size of the context window

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


In [27]:
eval_batch_size = 32  # Define evaluation batch size

@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_batch_size)
        for k in range(eval_batch_size):
            xb, yb = get_batch(split)
            logits, loss = m(xb, yb)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [ ]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias = False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x) 
        q = self.query(x)

        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        # perform the weighted aggregation of the values
        v = self.value(x)
        out = wei @ v
        return out

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList((Head(head_size) for _ in range(n_heads)))
        self.proj = nn.Linear(n_embd, n_embd)  # Projection layer to combine heads
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)  # Concatenate outputs from all heads
        out = self.proj(out)  # Project the concatenated output
        return out  # Return the final output after projection

In [ ]:
class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd),
            nn.ReLU(),
            nn.Linear(n_embd, n_embd),  # Output layer to match the input dimension
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [59]:
class Block(nn.Module):

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)  # For reproducibility
n_embd = 384  # Size of the embedding vectors

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*Block(n_embd, n_head = n_head) for _ in range(n_layer))  # Stack 3 blocks
        self.ln_f = nn.LayerNorm(n_embd)  # Final layer normalization
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T))
        x = tok_emb + pos_emb  # (B, T, C)
        x = self.blocks(x)
        x = self.ln_f(x)  # Apply final layer normalization
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T) 
            loss = F.cross_entropy(logits, targets)
            return logits, loss
        else:
            return logits, None
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:] # Get the last block_size tokens
            logits, loss = self(idx_cond) # predict next token
            logits = logits[:, -1, :] # focus on the last time step
            probs = F.softmax(logits, dim = -1) # convert to probabilities
            idx_next = torch.multinomial(probs, num_samples = 1) # sample from the distribution
            idx = torch.cat((idx, idx_next), dim=1) # append sampled index to the input
        return idx

m = BigramLanguageModel()
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)



In [ ]:
batch_size = 64
for step in range(5000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)  # Forward pass
    optimizer.zero_grad(set_to_none=True)  # Zero gradients
    loss.backward()  # Backward pass
    optimizer.step()  # Update parameters

    if step % 100 == 0:
        losses = estimate_loss()
        print(f"Step {step}, Train Loss: {losses['train']:.4f}, Val Loss: {losses['val']:.4f}")

Step 0, Train Loss: 4.4038, Val Loss: 4.4331
Step 100, Train Loss: 2.9025, Val Loss: 2.9542
Step 200, Train Loss: 2.6761, Val Loss: 2.6841
Step 300, Train Loss: 2.5400, Val Loss: 2.6125
Step 400, Train Loss: 2.4814, Val Loss: 2.4969
Step 500, Train Loss: 2.4528, Val Loss: 2.4460
Step 600, Train Loss: 2.4306, Val Loss: 2.4327
Step 700, Train Loss: 2.3956, Val Loss: 2.4063
Step 800, Train Loss: 2.3301, Val Loss: 2.3537
Step 900, Train Loss: 2.3514, Val Loss: 2.3402
Step 1000, Train Loss: 2.3334, Val Loss: 2.3126
Step 1100, Train Loss: 2.3145, Val Loss: 2.2848
Step 1200, Train Loss: 2.2828, Val Loss: 2.2863
Step 1300, Train Loss: 2.2957, Val Loss: 2.2930
Step 1400, Train Loss: 2.2600, Val Loss: 2.2545
Step 1500, Train Loss: 2.2734, Val Loss: 2.2604
Step 1600, Train Loss: 2.2309, Val Loss: 2.2637
Step 1700, Train Loss: 2.2266, Val Loss: 2.2693
Step 1800, Train Loss: 2.2318, Val Loss: 2.2264
Step 1900, Train Loss: 2.2408, Val Loss: 2.2251
Step 2000, Train Loss: 2.2126, Val Loss: 2.2319
Step

In [58]:
idx = torch.zeros((1,1), dtype=torch.long)  # Start with a single token
print(decode(m.generate(idx, max_new_tokens=1000)[0].tolist()))  # Generate text and decode it


Than many I may Cand time, a bewhive sair'd clotlyeath on citll himms to haldeceath.

PORKINGHERS ENEf Hax Gottin, when and my oupt art,
That An blatt, a my not says gabloat-gorn, for fortwery, will stereck: weell off vilm det plat night,
Laticame have with
CARD
I cain fily, wills asfuitlise thy and is the swern prog
Myworg; to trainfets for to my that Cuch I sham son I to and.
Aw and a douths
Of they pret this out vhome? oug hat to shaufistly, my look likeselfore:
Whe the bart'd with to by trougess:
I call shiand dor cake fill to douse fill liged-comlet, in em doock,
For are whit romman thou thim.
By I with me wither poinkse you entell.

SCAMAMEONTERNA:
For to to to shead, farve nosiorn on hand a to nomiters.
Bome.'

Thich jurn wor with;
Nor that rewnesfong my man letters:
And Lartly, his laguot  mest as 'poue crobly! jence parve of hise?
Gown flight, sulanfurgeay somee tone?

Pist loulds list: tut whordsssamy, hromakem welll is and of well my thou salbwredier:
Nouspron! gof they, be